In [2]:
pip install yfinance

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 48.2 MB/s eta 0:00:00:00:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 112.2/112.2 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.0/69.0 kB 17.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.6/115.6 kB 29.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 67.4 MB/s eta 0:00:00:00:01
  Created wheel for peewee: filename=peewee-3.17.0-cp310-cp310-linux_x86_64.whl size=275957 sha256=3eab0171e8df5d049c277890cf8124a2153840dcfcd1600509fd2b63909da254
  Stored in directory: /home/ec2-user/.cache/pip/wheels/c7/70/ad/212867e96e7004265a69c4aa5dcff00a95f547a67ba26e7e76
Successfully built peewee
Note: you may need to restart the kernel to use updated packages.


In [26]:
import json
import concurrent.futures
import urllib.request
from bs4 import BeautifulSoup
import pandas as pd
import requests
import logging
import boto3
import yfinance as yf
from io import BytesIO
from datetime import datetime

In [28]:
url = "https://www.slickcharts.com/sp500"
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'}

# urllib.request를 사용하여 HTML을 가져옴
req = urllib.request.Request(url, headers=headers)
response = urllib.request.urlopen(req)
html_content = response.read()

soup = BeautifulSoup(html_content, 'html.parser')
   
header_row = soup.find('table', {'class': 'table'}).find('tr')
header_columns = header_row.find_all(['th', 'td'])
header = [column.get_text(strip=True) for column in header_columns]

# 나머지 행은 데이터로 사용
data_rows = soup.find('table', {'class': 'table'}).find_all('tr')[1:]
data = []
for row in data_rows:
    columns = row.find_all(['th', 'td'])
    data.append([column.get_text(strip=True) for column in columns])

# DataFrame 생성
df = pd.DataFrame(data, columns=header)

In [32]:
df

,#,Company,Symbol,Portfolio%,Price,Chg,% Chg
0,1,Microsoft Corp,MSFT,7.32%,381.33,2.72,(0.72%)
1,2,Apple Inc.,AAPL,7.29%,190.02,0.23,(0.12%)
2,3,Amazon.com Inc,AMZN,3.48%,147.02,-0.71,(-0.48%)
3,4,Nvidia Corp,NVDA,3.08%,477.20,-5.22,(-1.08%)
4,5,Alphabet Inc. Class A,GOOGL,2.12%,136.64,0.23,(0.17%)
...,...,...,...,...,...,...,...
498,499,"Alaska Air Group, Inc.",ALK,0.01%,36.59,-0.25,(-0.67%)
499,500,"Solaredge Technologies, Inc.",SEDG,0.01%,76.78,-0.04,(-0.05%)
500,501,"Mohawk Industries, Inc.",MHK,0.01%,85.69,1.41,(1.67%)
501,502,Fox Corporation Class B,FOX,0.01%,28.04,-0.21,(-0.73%)


In [ ]:
s3 = boto3.client('s3')
BUCKET_NAME = 'sagemaker-crawling-s3-bucket'
DIR_NAME = 'data_500/'

url = "https://www.slickcharts.com/sp500"
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'}

# urllib.request를 사용하여 HTML을 가져옴
req = urllib.request.Request(url, headers=headers)
response = urllib.request.urlopen(req)
html_content = response.read()

soup = BeautifulSoup(html_content, 'html.parser')
   
header_row = soup.find('table', {'class': 'table'}).find('tr')
header_columns = header_row.find_all(['th', 'td'])
header = [column.get_text(strip=True) for column in header_columns]

# 나머지 행은 데이터로 사용
data_rows = soup.find('table', {'class': 'table'}).find_all('tr')[1:]
data = []
for row in data_rows:
    columns = row.find_all(['th', 'td'])
    data.append([column.get_text(strip=True) for column in columns])

# DataFrame 생성
df = pd.DataFrame(data, columns=header)
df['Symbol'] = df['Symbol'].str.replace('.', '-', regex=False)

for ticker_symbol in df['Symbol']:
    data_frame = yf.Ticker(ticker_symbol).history(period='max', interval='1d')
        
    # Save DataFrame to a CSV file
    csv_data = data_frame.to_csv(index=False)

    s3_object_key = f"{ticker_symbol}.csv"
    KEY = DIR_NAME + s3_object_key
    s3.put_object(Body=csv_data, Bucket=BUCKET_NAME, Key=KEY)

return {
    'statusCode': 200,
    'body' : 'Crawling and data upload success!'
}